In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

from https://www.analyticsvidhya.com/blog/2018/03/essentials-of-deep-learning-visualizing-convolutional-neural-networks/

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### loading datasets

In [ ]:
df = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
df.head()

In [ ]:
X = df.iloc[:, 1:]
y = df.iloc[:,0]
X.shape, y.shape

### creating a dataset class

In [ ]:
# my_transforms = transforms.ToTensor()
my_transforms = ""
class fashion_dataset(Dataset):
    def __init__(self,data_path="../input/fashionmnist/fashion-mnist_train.csv", transforms=my_transforms, subset_length=2000):
        if subset_length:
            self.input_df = pd.read_csv(data_path)[:subset_length]
        else :
            self.input_df = pd.read_csv(data_path)
        self.len = len(self.input_df)
        if transforms:
            self.transform = transforms
        else:
            self.transform = ""
        

        self.x  = torch.from_numpy(self.input_df.iloc[:,1:].values)
        self.x = self.x.reshape(self.x.size(0), 1, 28,28).float()
        
        self.y = torch.from_numpy(self.input_df.iloc[:,0].values).long()
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        x_item = self.x[index]
        y_item = self.y[index]
        
        if (self.transform):
            x_item = self.transform(x_item)
               
        return x_item, y_item 

In [ ]:
train_dataset = fashion_dataset()

In [ ]:
X, y = train_dataset[0]

print(X.shape, y)

import matplotlib.pyplot as plt

print(y)

img = X.permute(1,2,0)
plt.imshow(img)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=128,shuffle=True)

In [ ]:
examples = iter(train_dataloader)
example_data, example_targets = examples.next()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show() 

In [ ]:
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
 # Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
         # no activation and no softmax at the end
        return out 

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
len(train_dataloader)

In [ ]:
n_total_steps = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):  
         # origin shape: [100, 1, 28, 28]
         # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
         # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
         # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        #print(i)
        if (i+1) % 10 == 0:
             print (f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# need to visualise
also how is the model trained here